In [ ]:
import pandas as pd
import plotly.express as px
import ast

In [ ]:
country_coverage = pd.read_csv("../../data/places/coverage_by_country.csv")

In [ ]:
country_coverage

In [ ]:
# Drop "Unavailable" with no articles
country_coverage.drop(index=228, inplace=True)

## Standardization of names

In [ ]:
country_coverage.loc[country_coverage["country"] == "Congo-Brazzaville", "country"] = "Republic of the Congo"

In [ ]:
country_coverage.loc[country_coverage["country"] == "Palestine, State of", "country"] = "Palestine"

In [ ]:
country_coverage.loc[country_coverage["country"] == "Côte d'Ivoire", "country"] = "Ivory Coast"

In [ ]:
country_coverage.loc[country_coverage["country"] == "The Gambia", "country"] = "Gambia"

In [ ]:
country_coverage.loc[country_coverage["country"] == "Czechia", "country"] = "Czech Republic"

In [ ]:
country_coverage.loc[country_coverage["country"] == "Republic of the Congo", "iso_alpha3"] = "COG"

In [ ]:
country_coverage.loc[country_coverage["country"] == "Congo, The Democratic Republic of the", "country"] = "The Democratic Republic of the Congo"

In [ ]:
country_coverage.loc[country_coverage["country"] == "Türkiye", "country"] = "Turkey"

In [ ]:
nyt = country_coverage[(country_coverage["country"] != "United States")]

In [ ]:
fig = px.choropleth(nyt, 
    locations='iso_alpha3', 
    color='count_of_articles_nyt',
    color_continuous_scale="Reds",
    range_color=(0, 1000),
    locationmode="ISO-3"
)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})


fig.show()

In [ ]:
zeit = country_coverage[(country_coverage["country"] != "Germany")]

In [ ]:
fig = px.choropleth(zeit, 
    locations='iso_alpha3', 
    color='count_of_articles_zeit',
    color_continuous_scale="Reds",
    range_color=(1, 1000),
    locationmode="ISO-3"
)

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

## Calc percentage and middle point to show delta

In [ ]:
country_coverage.loc[country_coverage["country"] == "Germany", "count_of_articles_zeit"] = 0

In [ ]:
country_coverage.loc[country_coverage["country"] == "United States", "count_of_articles_nyt"] = 0

In [ ]:
country_coverage

In [ ]:
country_coverage["perc_of_coverage_nyt"] =  country_coverage["count_of_articles_nyt"].apply(lambda x: x / country_coverage["count_of_articles_nyt"].sum() * 100)

In [ ]:
country_coverage["perc_of_coverage_zeit"] =  country_coverage["count_of_articles_zeit"].apply(lambda x: x / country_coverage["count_of_articles_zeit"].sum() * 100)

In [ ]:
country_coverage["middle_point"] = country_coverage["perc_of_coverage_nyt"] - country_coverage["perc_of_coverage_zeit"]

Negative values are Zeit, Positive values are the NYT

In [ ]:
country_coverage

## Assign countries to one or the other outlet

In [ ]:
covered_by_zeit = country_coverage[country_coverage["middle_point"] < 0]

In [ ]:
covered_by_zeit = covered_by_zeit[["country", "iso_alpha3", "middle_point", "count_of_articles_zeit", "count_of_articles_nyt"]]

In [ ]:
covered_by_zeit

In [ ]:
covered_by_nyt = country_coverage[country_coverage["middle_point"] > 0]

In [ ]:
covered_by_nyt = covered_by_nyt[["country", "iso_alpha3", "middle_point", "count_of_articles_nyt", "count_of_articles_zeit"]]

### Load continental outline to identify regions

In [ ]:
european_entities = pd.read_csv('../../input-data/utilities/europe.txt', names=['Country'])
asia_entities = pd.read_csv('../../input-data/utilities/asia.txt', names=['Country'])
africa_entities = pd.read_csv('../../input-data/utilities/africa.txt', names=['Country'])
n_america_entities = pd.read_csv('../../input-data/utilities/north-america.txt', names=['Country'])
s_america_entities = pd.read_csv('../../input-data/utilities/south-america.txt', names=['Country'])
oceania_entities = pd.read_csv('../../input-data/utilities/oceania.txt', names=['Country'])
ant_entities = pd.read_csv('../../input-data/utilities/antarctica.txt', names=['Country'])
other_entities = pd.read_csv('../../input-data/utilities/other.txt', names=['Country'])


In [ ]:
european_entities["Region"] = "Europe"
asia_entities["Region"] = "Asia"
africa_entities["Region"] = "Africa"
n_america_entities["Region"] = "North America"
s_america_entities["Region"] = "South America"
oceania_entities["Region"] = "Oceania"
ant_entities["Region"] = "Antarctica"
other_entities["Region"] = "Other"

In [ ]:
regions_and_countries = pd.concat([european_entities, asia_entities, africa_entities, n_america_entities, s_america_entities, oceania_entities, other_entities,ant_entities])

In [ ]:
regions_and_countries = regions_and_countries.rename(columns={"Country": "country"})

In [ ]:
regions_and_countries = regions_and_countries.map(lambda x: x.strip() if isinstance(x, str) else x)

In [ ]:
regions_and_countries

In [ ]:
regions_zeit = covered_by_zeit.merge(regions_and_countries, how="left", left_on="country", right_on="country")

In [ ]:
regions_zeit.drop(index=83, inplace=True)

In [ ]:
regions_zeit.drop(index=40, inplace=True)

In [ ]:
regions_zeit["outlet"] = "Zeit"

In [ ]:
regions_nyt = covered_by_nyt.merge(regions_and_countries, how="left", left_on="country", right_on="country")

In [ ]:
regions_nyt["outlet"] = "NYT"

In [ ]:
regions_nyt = regions_nyt[['country', 'iso_alpha3', 'middle_point', 'count_of_articles_zeit', 'count_of_articles_nyt', 'Region', 'outlet']]

In [ ]:
regions_nyt.columns

In [ ]:
regions_zeit.columns

In [ ]:
coverage_by_outlet_and_region = pd.concat([regions_zeit, regions_nyt])

### Create a nested JSON for Voronoi

In [ ]:
out = coverage_by_outlet_and_region.groupby(["outlet","Region"])[['country', 'iso_alpha3', 'middle_point', 'count_of_articles_zeit', 'count_of_articles_nyt']].apply(lambda x: x.to_dict('records')).reset_index(name='countries')

In [ ]:
out = out.groupby(["outlet"])[['Region', 'countries']].apply(lambda x: x.to_dict('records')).reset_index(name='regions')

In [ ]:
out

In [ ]:
out.to_json("../../data/places/coverage_by_region.json", orient="records")

In [ ]:
fig = px.choropleth(country_coverage, 
    locations='iso_alpha3', 
    color='middle_point',
    color_continuous_scale="PiYG",
    range_color=(-0.01, 0.01),
    locationmode="ISO-3"
)

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

fig.show()

In [ ]:
country_coverage.to_csv("../../data/places/coverage_by_country.csv", index=False)

## Retrieve related categories for each location

### NYT

In [ ]:
full_year = pd.read_csv("../../input-data/temp-data.csv")

In [ ]:
country_coverage

In [ ]:
zeit_full_year = pd.read_csv("../../input-data/zeit-temp-data.csv")

In [ ]:
zeit_full_year

In [ ]:
full_year['keywords'] = full_year['keywords'].apply(ast.literal_eval)

In [ ]:
zeit_full_year = zeit_full_year[zeit_full_year['keywords'].notna()]

In [ ]:
zeit_full_year['keywords'] = zeit_full_year['keywords'].apply(ast.literal_eval)

In [ ]:
zeit_full_year_exploded = zeit_full_year.explode("keywords")

In [ ]:
zeit_full_year_exploded = zeit_full_year_exploded.rename(columns={"uri": "_id"})

In [ ]:
zeit_full_year_exploded

In [ ]:
zeit_full_year_exploded = zeit_full_year_exploded[["_id", "date", "url", "keywords"]]

In [ ]:
zeit_full_year_exploded.rename(columns={"uri": "_id"}, inplace=True)

In [ ]:
zeit_full_year_exploded

In [ ]:
full_year_exploded = full_year.explode("keywords")

In [ ]:
country_coverage["ids_of_articles_nyt"] = country_coverage["ids_of_articles_nyt"].apply(ast.literal_eval)

In [ ]:
country_coverage["ids_of_articles_zeit"] = country_coverage["ids_of_articles_zeit"].apply(ast.literal_eval)

In [ ]:
countries = country_coverage["country"].unique()

In [ ]:
list_of_keywords_dicts = []
for country in countries:
    country_selection = country_coverage[country_coverage["country"] == country]
    nyt_ids = country_selection["ids_of_articles_nyt"].values
    nyt_ids = nyt_ids[0]
    keywords_per_country = []
    
    for id in nyt_ids:
        article = full_year_exploded[full_year_exploded["_id"] == id]
        keywords_list = article["keywords"]
        list_of_keywords_per_article = list(keywords_list.apply(lambda x: x.get("value")))
        keywords_per_country.extend(list_of_keywords_per_article)

    my_dict = {i:{keywords_per_country.count(i)} for i in keywords_per_country}
    list_of_keywords_dicts.append(my_dict)
    print(len(list_of_keywords_dicts))

In [ ]:
country_coverage["keywords_nyt"] = list_of_keywords_dicts

In [ ]:
country_coverage

In [ ]:
country_coverage

In [ ]:
zeit_full_year_exploded

In [ ]:
list_of_zeit_keywords_dicts = []
for country in countries:
    print(country)
    country_selection = country_coverage[country_coverage["country"] == country]
    ids = country_selection["ids_of_articles_zeit"].values
    print(country_selection["ids_of_articles_zeit"].values)
    ids = ids[0]  # Assuming ids[0] is already a list
    zeit_keywords_per_country = []
    for id in ids:  # No need for eval() here
        article = zeit_full_year_exploded[zeit_full_year_exploded["_id"] == id]
        keyword = article["keywords"].values
        zeit_keywords_per_country.extend(keyword)
    
    my_dict = {i: zeit_keywords_per_country.count(i) for i in zeit_keywords_per_country}
    print("length of current dict", len(my_dict.keys()))
    list_of_zeit_keywords_dicts.append(my_dict)
    print("all dicts", len(list_of_zeit_keywords_dicts))

In [ ]:
country_coverage["keywords_zeit"] = list_of_zeit_keywords_dicts
country_coverage

In [ ]:
country_coverage["keywords_zeit"] = country_coverage["keywords_zeit"].apply(lambda x:{key: value for key, value in sorted(x.items(), key=lambda item: item[1], reverse=True)})

In [ ]:
country_coverage["keywords_nyt"] = country_coverage["keywords_nyt"].apply(lambda x:{key: value for key, value in sorted(x.items(), key=lambda item: item[1], reverse=True)})

In [ ]:
country_coverage.to_csv("../../data/places/coverage_by_country.csv", index=False)